In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('./data/daum_movie_review.csv')
# rating이 6보다 작으면 0(부정), 6 이상이면 긍정으로 라벨 생성
y =[0 if rate < 6 else 1 for rate in df.rating]
# 데이터셋을 학습, 검증, 평가 데이터셋으로 분리
X_train_val, X_test, y_train_val, y_test = train_test_split(df.review.tolist(), y, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=0)

print('#Train set size:', len(X_train))
print('#Validation set size:', len(X_val))
print('#Test set size:', len(X_test))

#Train set size: 8282
#Validation set size: 2761
#Test set size: 3682


In [26]:
import torch
import evaluate
from torch.utils.data import Dataset

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits,labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

class OurDataset(Dataset):
    def __init__(self, inputs, labels, transform=None, target_transform=None):
        self.inputs = inputs
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: self.transform(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = self.target_transform(self.labels[idx])
        return item

In [27]:
metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [19]:
'''
class TqdmCallback(Callback):
    """
    A callback to display a progress bar using tqdm

    Parameters
    ----------
    tqdm_kwargs : dict, (optional)
        Any argument accepted by the tqdm constructor.
        See the `tqdm doc <https://tqdm.github.io/docs/tqdm/#__init__>`_.
        Will be forwarded to tqdm.

    Examples
    --------
    >>> import fsspec
    >>> from fsspec.callbacks import TqdmCallback
    >>> fs = fsspec.filesystem("memory")
    >>> path2distant_data = "/your-path"
    >>> fs.upload(
            ".",
            path2distant_data,
            recursive=True,
            callback=TqdmCallback(),
        )

    You can forward args to tqdm using the ``tqdm_kwargs`` parameter.

    >>> fs.upload(
            ".",
            path2distant_data,
            recursive=True,
            callback=TqdmCallback(tqdm_kwargs={"desc": "Your tqdm description"}),
        )
    """

    def __init__(self, tqdm_kwargs=None, *args, **kwargs):
        try:
            import tqdm

            self._tqdm = tqdm
        except ImportError as exce:
            raise ImportError(
                "Using TqdmCallback requires tqdm to be installed"
            ) from exce

        self._tqdm_kwargs = tqdm_kwargs or {}
        super().__init__(*args, **kwargs)

    def set_size(self, size):
        self.tqdm = self._tqdm.tqdm(total=size, **self._tqdm_kwargs)


    def relative_update(self, inc=1):
        self.tqdm.update(inc)


    def __del__(self):
        self.tqdm.close()
        self.tqdm = None

'''

'\nclass TqdmCallback(Callback):\n    """\n    A callback to display a progress bar using tqdm\n\n    Parameters\n    ----------\n    tqdm_kwargs : dict, (optional)\n        Any argument accepted by the tqdm constructor.\n        See the `tqdm doc <https://tqdm.github.io/docs/tqdm/#__init__>`_.\n        Will be forwarded to tqdm.\n\n    Examples\n    --------\n    >>> import fsspec\n    >>> from fsspec.callbacks import TqdmCallback\n    >>> fs = fsspec.filesystem("memory")\n    >>> path2distant_data = "/your-path"\n    >>> fs.upload(\n            ".",\n            path2distant_data,\n            recursive=True,\n            callback=TqdmCallback(),\n        )\n\n    You can forward args to tqdm using the ``tqdm_kwargs`` parameter.\n\n    >>> fs.upload(\n            ".",\n            path2distant_data,\n            recursive=True,\n            callback=TqdmCallback(tqdm_kwargs={"desc": "Your tqdm description"}),\n        )\n    """\n\n    def __init__(self, tqdm_kwargs=None, *args, **kw

In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
print(tokenizer.tokenize("안녕하세요. 반갑습니다."))
inputs = tokenizer("안녕하세요. 반갑습니다.")
print(inputs)

loading file vocab.txt from cache at /Users/chance/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/chance/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json
loading configuration file config.json from cache at /Users/chance/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "interm

['안', '##녕', '##하', '##세', '##요', '.', '반', '##갑', '##습', '##니다', '.']
{'input_ids': [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [21]:
inputs.items()

dict_items([('input_ids', [101, 9521, 118741, 35506, 24982, 48549, 119, 9321, 118610, 119081, 48345, 119, 102]), ('token_type_ids', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('attention_mask', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])])

In [22]:
import os
os.getcwd()

'/Users/chance/Desktop/Coding/TextMining'

In [23]:
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

# 토큰화
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors="pt")
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")

# Dataset 생성
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)

# bert-base-multilingual-cased 사전학습 모형으로부터 분류기 모형을 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

loading configuration file config.json from cache at /Users/chance/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

loading weights file model.safet

In [24]:
# Trainer에서 사용할 하이퍼 파라미터 지정
training_args = TrainingArguments(
    output_dir='./results',         # 모형 예측이나 체크포인트 출력 폴더, 반드시 필요함
    num_train_epochs=2,             # 학습 에포크 수
    evaluation_strategy="steps",    # 에포크마다 검증 데이터셋에 대한 평가 지표를 출력
    eval_steps=500,
    per_device_train_batch_size=8,  # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=16,  # 평가에 사용할 배치 사이즈
    warmup_steps=200,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,              # strength of weight decay
)

# trainer 객체 생성
trainer = Trainer(
    model = model,                  # 학습할 모형
    args=training_args,             # 위에서 정의한 학습 매개변수
    train_dataset=train_dataset,    # 학습 데이터셋
    eval_dataset=val_dataset,       # 검증 데이터셋
    compute_metrics=compute_metrics,
)

# 미세조정학습 실행
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


TypeError: Instance and class checks can only be used with @runtime_checkable protocols